# Attempt for Curiosity of Physics Informed Machine Learning for Spin Tracking

In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numba

In [5]:
@numba.jit
def vecLen(a):
	return np.sqrt(np.sum(np.square(a)))

@numba.jit
def norm(a):
	return a/vecLen(a)

@numba.jit
def BPrime(B, E, v, gamma, c):
	return gamma * B + (1-gamma)*np.dot(B, v)*v/np.dot(v)-gamma/c**2.0*np.cross(v, E)

@numba.jit
def WT(gamma, c, a, v):
	return gamma**2.0/(c**2.0 * (gamma+1))*np.cross(a, v)

@numba.jit
def updateSpin(s, v, B0, E0, consts):
	s = norm(s)
	#this is the base iteration function, the idea is to shift all the values by one step each time this is called
	B = B0+np.cross(E0, v/89875517873681764.0) #big number is the speed of light squared in m^2/s^2
	#update the spin
	crossVal = consts['gamma']*B+2.0*(consts['dn']/consts['hbar'])*E0
	s = np.cross(s, crossVal)
	return s

# Finds value of y for a given x using step size h
@numba.jit
def rungeKutta(function, rkA, rkB, rkC, initSpin, h, numIter, V, B0, E0, consts):
	# Count number of iterations using step size or
	# step height h
	#how many k calculations are there
	y0 = np.copy(initSpin)
	s = len(rkC)
	allSpins = None
	allSpins = [y0]
	for n in range(numIter):
		k = [function(y0, v, B0, E0, consts)]
		for i in range(1, s):
			temp = np.zeros(y0.shape)
			for j in range(1, i):
				temp+=rkA[i][j]*k[j]
			k.append(function(y0+h*temp, v, B0, E0, consts))
		t = np.zeros(y0.shape)
		for i in range(s):
			t+=rkB[i]*k[i]
		y0 = y0 + h*t
		y0 = norm(y0)
		allSpins.append(y0)
	return allSpins

def forwardEuler():
	rkA = np.array([[0]])
	rkB = np.array([1])
	rkC = np.array([0])
	return rkA, rkB, rkC

def midpointMethod():
	rkA = np.array([[0, 0], [0.5, 0]])
	rkB = np.array([0, 1])
	rkC = np.array([0, 0.5])
	return rkA, rkB, rkC

def RK3():
	rkA = np.array([[0, 0, 0], [0.5, 0, 0], [-1, 2, 0]])
	rkB = np.array([1.0/6.0, 2.0/3.0, 1.0/6.0])
	rkC = np.array([0.0, 0.5, 1.0])
	return rkA, rkB, rkC

def RK4():
	rkA = np.array([[0, 0, 0, 0], [0.5, 0, 0, 0], [0, 0.5, 0, 0], [0, 0, 1, 0]])
	rkB = np.array([1.0/6.0, 1.0/3.0, 1.0/3.0, 1.0/6.0])
	rkC = np.array([0, 0.5, 0.5, 1.0])
	return rkA, rkB, rkC

def RK45():
	rkA = np.array([[0, 0, 0, 0, 0], 
		   [1/4, 0, 0, 0, 0],
		   [3/32, 9/32, 0, 0, 0],
		   [1932/2197, -7200/2197, 7296/2197, 0, 0],
		   [439/216, -8, 3680/513, -845/4140, 0],
		   [-8/27, 2, -3544/2565, 1859/4104, -11/40]])
	rkB = np.array([16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55])
	rkC = np.array([0, 1/4, 3/8, 12/13, 1, 1/2])
	return rkA, rkB, rkC

def CashKarp6_4_5():
	rkA = np.array([[0, 0, 0, 0, 0, 0],
		   [1/5, 0, 0, 0, 0, 0],
		   [3/40, 9/40, 0, 0, 0, 0],
		   [3/10, -9/10, 6/5, 0, 0, 0],
		   [-11/54, 5/2, -70/27, 35/2, 0, 0],
		   [1631/55296, 175/512, 575/13824, 44275/110592, 253/4096, 0]])
	rkB = np.array([37/378, 0, 250/621, 125/594, 0, 512/1771])
	rkC = np.array([0, 1/5, 3/10, 3/5, 1, 7/8])
	return rkA, rkB, rkC

In [6]:
precision = np.float64
neutType = np.dtype([('x', precision, 3), ('v', precision, 3), ('s', precision, 3)])

constType = np.dtype([('m', precision), ('dn', precision), ('gamma', precision), ('hbar', precision)])
consts = np.array(1, dtype=constType)
consts['m']=1.674927485E-27 #units of kg
consts['dn'] = 0.0
consts['gamma'] = -1.83247171E8
consts['hbar'] = 1.054571817E-34

#E = np.array([0, 0, 75.0E5], dtype=precision)
E = np.array([0, 0, 0], dtype=precision)
B = np.array([0, 0, 30.0E-7], dtype=precision)
s = np.array([1, 0, 0], dtype=precision)
v = np.array([0, 0, 0], dtype=precision)

In [7]:
#initial runs to get the JIT compilation out of the way
h = 1
num = 1
rungeKutta(updateSpin, *forwardEuler(), np.copy(s), h, num, v, B, E, consts)
rungeKutta(updateSpin, *midpointMethod(), np.copy(s), h, num, v, B, E, consts)
rungeKutta(updateSpin, *RK3(), np.copy(s), h, num, v, B, E, consts)
rungeKutta(updateSpin, *RK4(), np.copy(s), h, num, v, B, E, consts)
rungeKutta(updateSpin, *RK45(), np.copy(s), h, num, v, B, E, consts)
rungeKutta(updateSpin, *CashKarp6_4_5(), np.copy(s), h, num, v, B, E, consts)
None

In [22]:
w = consts['gamma']*vecLen(B) #calculate the frequency
period = abs(2.0*np.pi/w)
perPeriod = 10000
numPeriod = 10
timeScale = np.linspace(0, numPeriod*period, numPeriod*perPeriod, endpoint=False)
h = np.diff(timeScale)[0]

spinsEuler = np.array(rungeKutta(updateSpin, *forwardEuler(), np.copy(s), h, len(timeScale), v, B, E, consts))[:-1]

In [30]:
# Neural network
act = tf.keras.layers.ReLU()
nn_sv = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation=act, input_shape=(1,)),
  tf.keras.layers.Dense(10, activation=act),
  tf.keras.layers.Dense(1,activation='linear')])

In [31]:
loss_sv = tf.keras.losses.MeanSquaredError()
optimizer_sv = tf.keras.optimizers.Adam(learning_rate=0.001)
nn_sv.compile(optimizer=optimizer_sv, loss=loss_sv)

# Training
results_sv = nn_sv.fit(timeScale, spinsEuler, epochs=5, batch_size= 5, verbose=1)

Epoch 1/5
20000/20000 [==============================] - 45s 2ms/step - loss: 0.3333
Epoch 2/5
20000/20000 [==============================] - 45s 2ms/step - loss: 0.3332
Epoch 3/5
20000/20000 [==============================] - 45s 2ms/step - loss: 0.3332
Epoch 4/5
20000/20000 [==============================] - 45s 2ms/step - loss: 0.3331
Epoch 5/5
20000/20000 [==============================] - 44s 2ms/step - loss: 0.3332


In [32]:
predictions = nn_sv.predict(timeScale)

3125/3125 [==============================] - 7s 2ms/step
